In [28]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(reshape2))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(plyr))
source('/home/jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

In [1]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/SF4_meta_data-2024-05-05.csv")
meta_data$ageAtEnrollment[meta_data$ageAtEnrollment == '89+'] <- '90'
meta_data$ageAtEnrollment<-as.numeric(meta_data$ageAtEnrollment)
meta_data<-meta_data[!is.na(meta_data$ageAtEnrollment),]
meta_data$Ages<-meta_data$ageAtEnrollment
meta_data <- meta_data[!duplicated(meta_data$sample.sampleKitGuid, fromLast = TRUE), ]
rownames(meta_data)<-meta_data$sample.sampleKitGuid

In [3]:
file_list<-paste0("/home/jupyter/BRI_Figures/Dataset/scRNA/SF4/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/", meta_data$pbmc_sample_id, ".csv")

In [24]:
df_list<-read_pseudobulk_expression(file_list, mc_cores = 60)

[1] "Total reading time: 17.463 seconds"
[1] "The length of the list matches the length of the input path."


In [44]:
gene_list<-list(c("IRF3"),c("ID3"),
                c("LGALS1"),c("GZMK"))
names(gene_list)<-c("Core naive CD4 T cell","Core naive CD8 T cell",
                    "CM CD4 T cell","GZMB- CD27+ EM CD4 T cell")

In [45]:
unlist(gene_list[1])

Core naive CD4 T cell 
               "IRF3"

In [46]:
expression_list <- list()

for (i in seq_along(gene_list)) {
  gene_names <- unlist(gene_list[[i]])
  cell_type <- names(gene_list)[i]
  
  df <- filter_genes_and_celltype(df_list, gene_names, cell_type, longformat = TRUE)
  df <- left_join(df, meta_data)
  
  expression_list[[cell_type]] <- df
}

Joining with `by = join_by(pbmc_sample_id)`
Warning message in FUN(X[[i]], ...):
“Cell type Core naive CD8 T cell not found in the dataframe.”
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`


In [47]:
df<-do.call(rbind,expression_list)

In [48]:
write.csv(df,'Up_Selected_Genes.csv')